In [ ]:
!pip install numpy pandas matplotlib scipy scikit-learn spei seaborn pygeohydro pynhd

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import scipy.stats as scs
from scipy.stats import genextreme, pearson3
from scipy.optimize import curve_fit
from scipy.stats import genextreme as gev
from scipy.stats import pearson3
from sklearn.preprocessing import StandardScaler
import spei as si
import calendar


In [ ]:
# Get streamflow for your gage using pygeohydro and the NWIS
from pygeohydro import NWIS

gage_id ="08279500"
site = gage_id

nwis = NWIS()
info = nwis.get_info({"site": gage_id}, nhd_info=True)
area_sqm = info.nhd_areasqkm.values[0] * 1e6

flow = nwis.get_streamflow(gage_id, dates= ('1900-01-01', '2023-12-31'))
flow.index = pd.to_datetime(flow.index.date)
flow.head()


In [ ]:
flow.plot()

In [ ]:
# Make the figure
fig, ax = plt.subplots()

# Should we use log flow or regular flow?
use_log_flow = True
if use_log_flow:
    data = np.log(flow)
    y_label = 'Log Flow (cfs)'
else:
    data = flow
    y_label = 'Flow (cfs)'

# Use SeaBorn to make a histogram
sns.histplot(data, ax = ax, 
             color = 'cornflowerblue', 
             bins = 100, kde=True)

# Set the x and y axes labels
plt.ylabel('Number of times the flow was observed')
plt.xlabel(y_label)
plt.show()

In [ ]:
def plot_monthly_flow(flow_df, 
                      plot_type='box', 
                      color_palette='viridis', 
                      figsize=(12, 6),
                      title='Monthly Flow Distribution',
                      ylabel='Flow',
                      show_points=True,
                      point_color='red',
                      point_alpha=0.5):
    """
    Create a monthly flow plot with customization options.
    
    Parameters:
    - flow_df: DataFrame with datetime index and 'flow' column
    - plot_type: 'box' or 'violin'
    - color_palette: color palette for the plot
    - figsize: tuple for figure size
    - title: string for plot title
    - ylabel: string for y-axis label
    - show_points: boolean to show individual points
    - point_color: color of individual points
    - point_alpha: transparency of individual points
    
    Returns:
    - matplotlib figure

    # Example usage:
    # fig = plot_monthly_flow(flow_df, plot_type='violin', color_palette='Set3', show_points=True)
    # fig.show()
    """
    # Aggregate daily data to monthly
    monthly_flow = flow_df.resample('M').agg(['mean', 'min', 'max', 'median'])
    monthly_flow.columns = monthly_flow.columns.droplevel()
    monthly_flow['month'] = monthly_flow.index.month
    
    # Create plot
    plt.figure(figsize=figsize)
    
    if plot_type == 'box':
        sns.boxplot(x='month', y='mean', data=monthly_flow, palette=color_palette)
    elif plot_type == 'violin':
        sns.violinplot(x='month', y='mean', data=monthly_flow, palette=color_palette)
    else:
        raise ValueError("plot_type must be 'box' or 'violin'")
    
    if show_points:
        sns.stripplot(x='month', y='mean', data=monthly_flow, 
                      color=point_color, alpha=point_alpha, jitter=True)
    
    plt.title(title)
    plt.xlabel('Month')
    plt.ylabel(ylabel)
    plt.xticks(range(12), calendar.month_abbr[1:13])
    
    return plt


### Now use the function to make the plot!

plot_monthly_flow(flow, 
                  show_points=False)

## Drought detection

The next set of functions are used to measure and identify drought events. 

In [ ]:
def get_drought_metrics(ssi):
    """Get drought start and end dates, magnitude, severity, and duration.

    Args:
        ssi (pd.Series): Array of SSI values.  

    Returns:
        pd.DataFrame: DataFrame containing all drought metrics for each drought period.
    """
    
    drought_data = {}
    drought_counter = 0
    in_critical_drought = False
    drought_days = []

    for ind in range(len(ssi)):
        if ssi.values[ind] < 0:
            drought_days.append(ind)
            
            if ssi.values[ind] <= -1:
                in_critical_drought = True
        else:
            # Record drought info once it ends
            if in_critical_drought:
                drought_counter += 1
                drought_data[drought_counter] = {
                    'start':ssi.index[drought_days[0]],
                    'end': ssi.index[drought_days[-1]],
                    'duration': len(drought_days),
                    'magnitude': sum(ssi.values[drought_days]),
                    'severity': min(ssi.values[drought_days])
                }
            
            # Reset counters
            in_critical_drought = False
            drought_days = [] 

    drought_metrics = pd.DataFrame(drought_data).transpose()
    return drought_metrics

This function is going to look at the droughts, and then calculate a few different variables of interest... 

In [ ]:

def drought_metric_scatter_plot(drought_metrics):
    fig, ax = plt.subplots(figsize = (7,6))
    p = ax.scatter(drought_metrics['severity'], -drought_metrics['magnitude'],
            c= drought_metrics['duration'], cmap = 'viridis_r', s=100)

    plt.colorbar(p).set_label(label = 'Drought Duration (days)',size=15)
    plt.xlabel(r'Severity ($Minimum SSI_{6}$)', fontsize = 15)
    plt.ylabel(r'Magnitude (Acc. Deficit)', fontsize = 15)
    plt.title(f'Historic Droughts', fontsize = 16)
    plt.legend(fontsize=14)
    plt.tight_layout()
    plt.show()
    return


The next function is used to make a fancy timeseries plot... don't worry about how it works for now. 

In [ ]:
def plot_ssi(ssi, bound = 3.0, figsize= (8, 4), ax = None, gradient = False):
    """Plot the standardized index values as a time series. 
    Modified from https://github.com/martinvonk/SPEI/blob/main/src/spei/plot.py

    Parameters
    ----------
    si : pandas.Series
        Series of the standardized index
    bound : int, optional
        Maximum and minimum ylim of plot
    figsize : tuple, optional
        Figure size, by default (8, 4)
    ax : matplotlib.Axes, optional
        Axes handle, by default None which create a new axes

    Returns
    -------
    matplotlib.Axes
        Axes handle
    """
    if ax is None:
        _, ax = plt.subplots(figsize=figsize)

    ax.plot(ssi, color="k", lw = 1, label="SSI")
    ax.axhline(0, linestyle="--", color="k")

    nmin = -bound
    nmax = bound
    # Classify droughts
    in_drought = False
    in_critical_drought = False

    droughts = np.zeros_like(ssi.values)
    drought_days = []
    drought_counter = 0
    for ind in range(len(droughts)):
        if ssi.values[ind] < 0:
            in_drought = True
            drought_days.append(ind)
            
            if ssi.values[ind] <= -1:
                in_critical_drought = True
                droughts[drought_days] = 1
        else:
            in_drought = False
            in_critical_drought = False
            drought_days = [] 
            drought_counter = 0
        
    if gradient:
        droughts = ssi.to_numpy(dtype=float, copy=True)
        droughts[droughts > 0] = 0
        nodroughts = ssi.to_numpy(dtype=float, copy=True)
        nodroughts[nodroughts < 0] = 0
        x, y = np.meshgrid(ssi.index, np.linspace(nmin, nmax, 100))
        ax.contourf(
            x, y, y, cmap=plt.get_cmap("seismic_r"), levels=np.linspace(nmin, nmax, 100)
        )
        ax.fill_between(x=ssi.index, y1=droughts, y2=nmin, color="w")
        ax.fill_between(x=ssi.index, y1=nodroughts, y2=nmax, color="w")
    else:
        ax.fill_between(x=ssi.index, y1=nmax, y2=nmin, where=(droughts>0), color='red', alpha=0.5, interpolate=False, label = 'Drought Period')
    ax.set_ylim(nmin, nmax)
    return ax

In [ ]:
# Calculate the rolling sum of flow

# Specify the window size (number of days)
ssi_window = 5 * 12 *30   # default: 2 years (24 months * 30 days)


rolling_sum = flow.rolling(window = ssi_window).sum().dropna()

# Calculate SSI values
ssi_values = si.ssfi(rolling_sum, dist = scs.gamma)

plot_ssi(ssi_values, figsize=(10,5))

Recall that when we calculate droughts we need to set the "rolling window" size... this is going to have an impact on the droughts that we find.  If the window is very small, then the droughts will be insignificant.  If the window is very big, then we will identify only very big droughts.  

In the next cell, we do the following:
1. Set the rolling window size
2. Calculate the rolling sum
3. Calculate the SSI
4. Plot the time periods which are identified as droughts.

In [ ]:
# Calculate the rolling sum of flow
ssi_window = "720D"
rolling_sum = flow.rolling(window = ssi_window).sum().dropna()

# Calculate SSI values
ssi_values = si.ssfi(rolling_sum, dist = scs.gamma)

plot_ssi(ssi_values)

In [ ]:
drought_metrics = get_drought_metrics(ssi_values)

drought_metric_scatter_plot(drought_metrics)

## Annual extremes

Now we will look at the annual MAX and MIN flow at your gage.  

Is there any change over time?

In [ ]:

def plot_annual_extremes(flow_df, 
                         figsize=(12, 6),
                         colors=('skyblue', 'salmon'),
                         line_styles=('-', '-'),
                         markers=('o', 's'),
                         title='Annual Minimum and Maximum Flows',
                         ylabel_min='Minimum Flow',
                         ylabel_max='Maximum Flow',
                         show_legend=True,
                         legend_location='best',
                         log_scale=True,
                         add_trendlines=False):
    """
    Create a dual-axis plot showing the lowest and highest flow for each year over time.
    
    Parameters:
    - flow_df: DataFrame with datetime index and 'flow' column
    - figsize: tuple for figure size
    - colors: tuple of colors for (min_flow, max_flow)
    - line_styles: tuple of line styles for (min_flow, max_flow)
    - markers: tuple of marker styles for (min_flow, max_flow)
    - title: string for plot title
    - ylabel_min: string for y-axis label for minimum flow
    - ylabel_max: string for y-axis label for maximum flow
    - show_legend: boolean to show legend
    - legend_location: string for legend position
    - add_trendlines: boolean to add trend lines
    
    Returns:
    - matplotlib figure
    """
    # Resample to annual minimum and maximum
    annual_extremes = flow_df.resample('Y').agg(['min', 'max'])
    annual_extremes.columns = annual_extremes.columns.droplevel()
    
    # Create plot with dual y-axes
    fig, ax1 = plt.subplots(figsize=figsize)
    ax2 = ax1.twinx()
    
    # Plot minimum flow on left axis
    lns1 = ax1.plot(annual_extremes.index, annual_extremes['min'], 
                    color=colors[0], linestyle=line_styles[0], marker=markers[0],
                    label='Annual Minimum')
    ax1.set_ylabel(ylabel_min, color=colors[0])
    ax1.tick_params(axis='y', labelcolor=colors[0])
    
    # Plot maximum flow on right axis
    lns2 = ax2.plot(annual_extremes.index, annual_extremes['max'], 
                    color=colors[1], linestyle=line_styles[1], marker=markers[1],
                    label='Annual Maximum')
    ax2.set_ylabel(ylabel_max, color=colors[1])
    ax2.tick_params(axis='y', labelcolor=colors[1])
    
    if add_trendlines:
        # Add trend lines
        sns.regplot(x=annual_extremes.index.astype(int), y=annual_extremes['min'], 
                    ax=ax1, scatter=False, color=colors[0], label='Min Trend')
        sns.regplot(x=annual_extremes.index.astype(int), y=annual_extremes['max'], 
                    ax=ax2, scatter=False, color=colors[1], label='Max Trend')
    
    ax1.set_title(title)
    ax1.set_xlabel('Year')
    

    if show_legend:
        lns = lns1 + lns2
        labs = [l.get_label() for l in lns]
        ax1.legend(lns, labs, loc=legend_location)
    if log_scale:
        ax1.set_yscale('log')
        ax2.set_yscale('log')

    plt.tight_layout()
    return plt

# Example usage:
# fig = plot_annual_extremes(flow_df, colors=('blue', 'red'), markers=('o', '^'), add_trendlines=True)
# fig.show()


plot_annual_extremes(flow)

## Flood events

The next figure will calculate the biggest flood each year, and then put them in rank order. 

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

def plot_ranked_floods(flow_df, 
                       flow_column,
                       figsize=(8, 8),
                       title='Ranked Annual Maximum Flood Events',
                       xlabel='Rank',
                       ylabel='Flow',
                       color_scheme='viridis',
                       show_colorbar=True,
                       annotate_points=True,
                       point_size=80,
                       point_alpha=0.75):
    """
    Create a rank plot of the biggest floods each year, with color indicating the year.
    
    Parameters:
    - flow_df: DataFrame with datetime index and a column containing flow data
    - flow_column: Name of the column in flow_df that contains the flow data
    - n_events: Number of top flood events to plot
    - figsize: tuple for figure size
    - title: string for plot title
    - xlabel: string for x-axis label
    - ylabel: string for y-axis label
    - color_scheme: colormap name or 'custom' for a water-themed gradient
    - show_colorbar: boolean to show colorbar
    - annotate_points: boolean to annotate points with year
    - point_size: size of scatter points
    
    Returns:
    - matplotlib figure
    """
    # Verify the flow column exists
    if flow_column not in flow_df.columns:
        raise ValueError(f"Column '{flow_column}' not found in the DataFrame. Available columns are: {', '.join(flow_df.columns)}")

    # Get annual maximum flows
    annual_max = flow_df.resample('Y').max()
    n_events = len(annual_max)

    # Sort and rank the floods
    ranked_floods = annual_max.sort_values(flow_column, ascending=True).head(n_events)
    ranked_floods['rank'] = range(1, len(ranked_floods) + 1)
    
    # Create plot
    fig, ax = plt.subplots(figsize=figsize)
    
    # Create custom colormap if specified
    if color_scheme == 'custom':
        colors = ['#08519c', '#3182bd', '#6baed6', '#bdd7e7', '#eff3ff']
        color_scheme = LinearSegmentedColormap.from_list("custom_blue", colors)
    
    # Create scatter plot
    scatter = ax.scatter(ranked_floods['rank'], ranked_floods[flow_column], 
                         c=ranked_floods.index.year, cmap=color_scheme, 
                         s=point_size, edgecolor='black', linewidth=1,
                         alpha=point_alpha)
    
    # Customize plot
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.grid(True, linestyle='--', alpha=0.7)
    
    # Add colorbar
    if show_colorbar:
        cbar = plt.colorbar(scatter)
        cbar.set_label('Year')
    
    # Annotate points with year
    if annotate_points:
        for idx, row in ranked_floods.iterrows():
            ax.annotate(str(idx.year), (row['rank'], row[flow_column]),
                        xytext=(5, 5), textcoords='offset points')
    
    plt.tight_layout()
    return plt

# Example usage:
# fig = plot_ranked_floods(flow_df, flow_column='discharge', n_events=25, color_scheme='custom', annotate_points=True)
# fig.show()

# Example usage:
# fig = plot_ranked_floods(flow_df, n_events=25, color_scheme='custom', annotate_points=True)
# fig.show()


plot_ranked_floods(flow, flow_column=flow.columns[0], annotate_points=False)

## Experimentation

Now that you've seen a few different plots, try experimenting with your own ideas!

Use the AI code feature to help me design and make the code.